## Interpreting coefficients in polynomial models

In [ ]:
import pandas as pd
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
from jupyprint import jupyprint, arraytex

- the slopes of polynomial terms are easy to "see" the effects of graphically...

- ...but trickier to interpret than "normal" slopes

- even a single-predictor model with a quadratic term is a multiple regression model, but the "one unit change while everything else is held constant" interpretation *cannot* be true

- if we have $x$ and $x^{2}$ in a model, as $x$ increases by 1 unit, so must $x^2$ (likewise for other polynomial terms)

- the expected change in the outcome variable for a 1-unit increase in the predictor **changes depending what level of the predictor we look at* e.g. it changes at different rates at low values of the predictor vs high values of the predictor

- let's investigate this further:

In [ ]:
cars_df = pd.read_csv('data/Auto.csv')

display(cars_df.head(20))

In [ ]:
import statsmodels.formula.api as smf

quadratic_mod = smf.ols('mpg ~ horsepower + I(horsepower**2)', data=cars_df).fit()

quadratic_mod.summary()

In [ ]:
def one_unit_difference_prediction(horsepower_score, return_diff=True):
    "A function to calculate the expected change in the outcome variable, at different values of the predictor."
    car_1_prediction = quadratic_mod.params['Intercept'] + quadratic_mod.params['horsepower']*horsepower_score + quadratic_mod.params['I(horsepower ** 2)']*horsepower_score**2
    car_2_prediction = quadratic_mod.params['Intercept'] + quadratic_mod.params['horsepower']*(horsepower_score+1) + quadratic_mod.params['I(horsepower ** 2)']*(horsepower_score+1)**2
    jupyprint("________________________________________")
    jupyprint(f"Prediction for `horsepower == {horsepower_score}` is {round(car_1_prediction, 2)}")
    jupyprint(f"Prediction for `horsepower == {horsepower_score+1}` is {round(car_2_prediction, 2)}")
    jupyprint(f"The difference is {round(car_2_prediction - car_1_prediction, 3)}")
    if return_diff == True:
        return round(car_2_prediction - car_1_prediction, 3)

one_unit_difference_prediction(150)

In [ ]:
differences = np.array([])
horsepower_range = np.arange(cars_df['horsepower'].min(), cars_df['horsepower'].max()+1, 10)
for score in horsepower_range:
    differences = np.append(differences, one_unit_difference_prediction(score))

We observe a nonconstant "expected change in the outcome variable at for a 1-unit change in the predictor" depending whether we are predicting for low values of the predictor or high values of the predictor:

In [ ]:
# nonconstant expected change per 1-unit increase in the predictor
plt.scatter(horsepower_range, differences)
plt.xlabel('Horsepower')
plt.ylabel('Difference in $\hat{y}$ Per One-Unit Change in Horsepower');

If you're so inclined, you can get the same information by doing some calculus with the slopes, *for a specific value of the predictor* (see [this stats.stackexchange post for more](https://stats.stackexchange.com/a/28750)):

In [ ]:
# expected 1-unit change in the outcome (compare this to the printout above, for the change between `horsepower == 226`
# and `horsepower == 227`
quadratic_mod.params['horsepower'] + 2*quadratic_mod.params['I(horsepower ** 2)'] * 226

# Polynomial terms in logistic regression models

In [ ]:
# code ported/adapted from R to python from here: https://book.stat420.org/logistic-regression.html

def simulated_quadratic_relationship(sample_size=50,
                                     predictor_abs_max=10,
                                     intercept = -1.5,
                                     linear_slope = 0.5,
                                     quadratic_slope = 0,
                                     cubic_slope = 0,
                                     quartic_slope = 0,
                                     quintic_slope = 0,
                                     sextic_slope = 0,
                                     septic_slope =0,
                                    show_log_odds=False):

    x = np.linspace(-predictor_abs_max, predictor_abs_max )
    x_points= np.random.uniform(-predictor_abs_max,predictor_abs_max , size = sample_size)
    linear_predictor = intercept + linear_slope * x
    linear_predictor_points = intercept + linear_slope * x_points
    for i, slope in enumerate([quadratic_slope, cubic_slope, quartic_slope, quintic_slope, sextic_slope, septic_slope]):
        linear_predictor += slope * x**(i+2)
        linear_predictor_points += slope * x_points**(i+2)

    p = np.exp(linear_predictor)/(1 + np.exp(linear_predictor))

    log_odds = np.log((p)/(1-p))

    p_points = np.exp(linear_predictor_points)/(1 + np.exp(linear_predictor_points))
    actual_y = np.array([])
    
    for i in np.arange(sample_size):
        actual_y = np.append(actual_y, np.random.binomial(1, p = p_points[i]))        

    if show_log_odds != False:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(x, p)
        plt.scatter(x_points, actual_y, color = 'red')
        plt.xlabel("Predictor")
        plt.ylim([0, 1])
        plt.ylabel("Probability")
    
        plt.subplot(1, 2, 2)
        plt.plot(x, log_odds)
        plt.xlabel("Predictor")
        plt.ylabel("Log Odds")

    else:
        plt.plot(x, p)
        plt.scatter(x_points, actual_y, color = 'red')
        plt.xlabel("Predictor")
        plt.ylim([0, 1])
        plt.ylabel("Probability")

interact(simulated_quadratic_relationship, predictor_abs_max =(10, 100),
                                     sample_size=(10, 1000, 1),
                                     intercept = (-1, 1, 0.1),
                                     linear_slope = (-2, 2, 0.001),
                                     quadratic_slope = (-1, 1, 0.001),
                                     cubic_slope = (-1, 1, 0.001),
                                     quartic_slope = (-0.01, 0.01, 0.001),
                                     quintic_slope = (-0.01, 0.01, 0.001),
                                     sextic_slope = (-0.001, 0.001, 0.0001),
                                     septic_slope =(-0.0001, 0.0001, 0.00001));